# To-Dos
- [ ] Keep overall crime per gemeente in final dataset; can then violent crime as a percent of total crime in each gemeente

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math

# Data prep

## Import crime data

In [3]:
## Data extracted from: https://data.politie.nl/portal.html?_la=nl&_catalog=Politie&tableId=47013NED&_theme=114
## On: 12-07-2025

crime_data = 'RegisteredCrimes_AllCrimes_Gemeenten_2024.csv'
metadata = 'AlleMisdrijven_Metadata.csv'

meta_df = pd.read_csv(metadata, sep=';', index_col=1, header=1)
meta_df

,ID,ShortTitle,Identifier,Summary,Modified,ReasonDelivery,ExplanatoryText,Language,Catalog,Frequency,Period,ShortDescription,Description,DefaultPresentation,DefaultSelection,GraphTypes,OutputStatus,Source,MetaDataModified,SearchPriority
Title,,,,,,,,,,,,,,,,,,,,
"Geregistreerde misdrijven en aangiften; soort misdrijf, gemeente 2025",0,Misdrijven en aangiften per gemeente,47013NED,"Geregistreerde misdrijven, aangiften en intern...",2025-06-16T02:00:00,ActualiseringBijzonder,NaN,nl,Politie,Permaand,2012-2025,\nDeze tabel bevat cijfers over het aantal ger...,INHOUDSOPGAVE\n\n1. Toelichting\n2. Definities...,ts=1749630168797&graphtype=Table&r=RegioS&k=To...,$filter=((SoortMisdrijf eq '0.0.0 ')) and ((Pe...,"Table,Bar,Map",Regulier,&copy; Politie,2025-06-16T02:00:00,2.0
NaN,DataProperties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,ID,ParentID,Type,Key,Title,Description,ReleasePolicy,Datatype,Unit,Decimals,Default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,NaN,Dimension,SoortMisdrijf,Soort misdrijf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,GeoDimension,RegioS,Regio's,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025 januari,2025MM01,NaN,Definitief,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025 februari,2025MM02,NaN,Definitief,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025 maart,2025MM03,NaN,Definitief,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
crime_df = pd.read_csv(crime_data, sep=';')
crime_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1
0,3492,0.0.0,GM1680,2012JJ00,925.0
1,3505,0.0.0,GM1680,2013JJ00,755.0
2,3518,0.0.0,GM1680,2014JJ00,664.0
3,3531,0.0.0,GM1680,2015JJ00,634.0
4,3544,0.0.0,GM1680,2016JJ00,616.0
...,...,...,...,...,...
263843,3847778,3.9.3,GM0999,2020JJ00,NaN
263844,3847791,3.9.3,GM0999,2021JJ00,NaN
263845,3847804,3.9.3,GM0999,2022JJ00,NaN
263846,3847817,3.9.3,GM0999,2023JJ00,NaN


## Extract crime metadata

In [6]:
print('Unique crime codes:')
print(crime_df['SoortMisdrijf'].unique())
print('Unqiue gemeente codes:')
print(crime_df['RegioS'].unique())
print('Unique year codes:')
print(crime_df['Perioden'].unique())

Unique crime codes:
['0.0.0 ' '1.1.1 ' '1.1.2 ' '1.2.1 ' '1.2.2 ' '1.2.3 ' '1.2.4 ' '1.2.5 '
 '1.3.1 ' '1.4.1 ' '1.4.2 ' '1.4.3 ' '1.4.4 ' '1.4.5 ' '1.4.6 ' '1.4.7 '
 '1.5.2 ' '1.6.1 ' '1.6.2 ' '1.6.3 ' '2.1.1 ' '2.2.1 ' '2.4.1 ' '2.4.2 '
 '2.5.1 ' '2.5.2 ' '2.6.1 ' '2.6.2 ' '2.6.3 ' '2.6.4 ' '2.6.5 ' '2.6.7 '
 '2.6.8 ' '2.6.9 ' '2.6.10' '2.6.11' '2.6.12' '2.6.13' '2.6.14' '2.7.2 '
 '2.7.3 ' '3.1.1 ' '3.1.2 ' '3.1.3 ' '3.2.1 ' '3.2.2 ' '3.3.2 ' '3.3.5 '
 '3.4.2 ' '3.5.2 ' '3.5.5 ' '3.6.4 ' '3.7.1 ' '3.7.2 ' '3.7.3 ' '3.7.4 '
 '3.9.1 ' '3.9.2 ' '3.9.3 ']
Unqiue gemeente codes:
['GM1680' 'GM0358' 'GM0197' 'GM0059' 'GM0482' 'GM0613' 'GM0361' 'GM0141'
 'GM0034' 'GM0484' 'GM1723' 'GM1959' 'GM0060' 'GM0307' 'GM0362' 'GM0363'
 'GM0200' 'GM0202' 'GM0106' 'GM0743' 'GM0744' 'GM0308' 'GM0489' 'GM0203'
 'GM0888' 'GM1954' 'GM0889' 'GM1945' 'GM1724' 'GM0893' 'GM0373' 'GM0748'
 'GM1859' 'GM1721' 'GM0753' 'GM0209' 'GM0375' 'GM0310' 'GM1728' 'GM0376'
 'GM0377' 'GM1901' 'GM0755' 'GM1681' 'GM0147' 'GM065

In [7]:
crime_codes = list(crime_df['SoortMisdrijf'].unique())
gemeente_codes = list(crime_df['RegioS'].unique())
year_codes = list(crime_df['Perioden'].unique())

In [8]:
crime_tuples = tuple(crime_codes)

crime_indices = []

for i in list(meta_df.index):
    if type(i) == float:
        continue
    elif i.startswith(crime_tuples):
        crime_indices.append(i)

crime_keys = []

for i in crime_indices:
    code_split = i.split(' ', maxsplit=1)
    crime_keys.append(code_split)


In [9]:
violent_crime = ['Zedendelicten','Moord, doodslag', 'Openlijk geweld (persoon)',
 'Bedreiging',
 'Mishandeling',
 'Straatroof',
 'Overval','Mensenhandel','Mensensmokkel', 'Kinderporno',
 'Kinderprostitutie']

In [10]:
crimekeys_df = pd.DataFrame(crime_keys,columns=['code','crime_name'])

violentcrime_df = crimekeys_df[crimekeys_df['crime_name'].isin(violent_crime)]

violentcrime_df.set_index(keys='code', inplace=True)

violentcrime_series = violentcrime_df['crime_name']

In [11]:
gmcodes_series = meta_df[meta_df['ID'].isin(gemeente_codes)]['ID']
yearcodes_series = meta_df[meta_df['ID'].isin(year_codes)]['ID']
gmcodes_series

Title
Aa en Hunze                    GM1680
Aalsmeer                       GM0358
Aalten                         GM0197
Achtkarspelen                  GM0059
Alblasserdam                   GM0482
                                ...  
Zwartewaterland                GM1896
Zwijndrecht                    GM0642
Zwolle                         GM0193
Buitenland                     GM0998
Gemeenten; niet in te delen    GM0999
Name: ID, Length: 344, dtype: object

In [12]:
violentcrime_dict = dict(violentcrime_series)
gm_dict = {}
for i, v in zip(gmcodes_series.index, gmcodes_series.values):
   gm_dict[v] = i

years_dict = {}
for i, v in zip(yearcodes_series.index, yearcodes_series.values):
    years_dict[v] = i


In [13]:
violentcrime_dict

{'1.4.1': 'Zedendelicten',
 '1.4.2': 'Moord, doodslag',
 '1.4.3': 'Openlijk geweld (persoon)',
 '1.4.4': 'Bedreiging',
 '1.4.5': 'Mishandeling',
 '1.4.6': 'Straatroof',
 '1.4.7': 'Overval',
 '1.6.3': 'Mensenhandel',
 '3.1.2': 'Mensensmokkel',
 '3.2.1': 'Kinderporno',
 '3.2.2': 'Kinderprostitutie'}

## Map crime, gemeente and year by codes to crime data

In [15]:
violentcrime_dict

{'1.4.1': 'Zedendelicten',
 '1.4.2': 'Moord, doodslag',
 '1.4.3': 'Openlijk geweld (persoon)',
 '1.4.4': 'Bedreiging',
 '1.4.5': 'Mishandeling',
 '1.4.6': 'Straatroof',
 '1.4.7': 'Overval',
 '1.6.3': 'Mensenhandel',
 '3.1.2': 'Mensensmokkel',
 '3.2.1': 'Kinderporno',
 '3.2.2': 'Kinderprostitutie'}

In [16]:
crime_df['SoortMisdrijf'] = crime_df['SoortMisdrijf'].apply(lambda x: x.strip())

In [17]:
crime_df['SoortMisdrijf'].unique()

array(['0.0.0', '1.1.1', '1.1.2', '1.2.1', '1.2.2', '1.2.3', '1.2.4',
       '1.2.5', '1.3.1', '1.4.1', '1.4.2', '1.4.3', '1.4.4', '1.4.5',
       '1.4.6', '1.4.7', '1.5.2', '1.6.1', '1.6.2', '1.6.3', '2.1.1',
       '2.2.1', '2.4.1', '2.4.2', '2.5.1', '2.5.2', '2.6.1', '2.6.2',
       '2.6.3', '2.6.4', '2.6.5', '2.6.7', '2.6.8', '2.6.9', '2.6.10',
       '2.6.11', '2.6.12', '2.6.13', '2.6.14', '2.7.2', '2.7.3', '3.1.1',
       '3.1.2', '3.1.3', '3.2.1', '3.2.2', '3.3.2', '3.3.5', '3.4.2',
       '3.5.2', '3.5.5', '3.6.4', '3.7.1', '3.7.2', '3.7.3', '3.7.4',
       '3.9.1', '3.9.2', '3.9.3'], dtype=object)

In [18]:
crime_df['crime_cat'] = crime_df['SoortMisdrijf'].map(violentcrime_dict)
crime_df['gemeente'] = crime_df['RegioS'].map(gm_dict)
crime_df['year'] = crime_df['Perioden'].map(years_dict)

In [19]:
crime_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1,crime_cat,gemeente,year
0,3492,0.0.0,GM1680,2012JJ00,925.0,NaN,Aa en Hunze,2012
1,3505,0.0.0,GM1680,2013JJ00,755.0,NaN,Aa en Hunze,2013
2,3518,0.0.0,GM1680,2014JJ00,664.0,NaN,Aa en Hunze,2014
3,3531,0.0.0,GM1680,2015JJ00,634.0,NaN,Aa en Hunze,2015
4,3544,0.0.0,GM1680,2016JJ00,616.0,NaN,Aa en Hunze,2016
...,...,...,...,...,...,...,...,...
263843,3847778,3.9.3,GM0999,2020JJ00,NaN,NaN,Gemeenten; niet in te delen,2020
263844,3847791,3.9.3,GM0999,2021JJ00,NaN,NaN,Gemeenten; niet in te delen,2021
263845,3847804,3.9.3,GM0999,2022JJ00,NaN,NaN,Gemeenten; niet in te delen,2022
263846,3847817,3.9.3,GM0999,2023JJ00,NaN,NaN,Gemeenten; niet in te delen,2023


In [20]:
crime_df['crime_cat'].unique()

array([nan, 'Zedendelicten', 'Moord, doodslag',
       'Openlijk geweld (persoon)', 'Bedreiging', 'Mishandeling',
       'Straatroof', 'Overval', 'Mensenhandel', 'Mensensmokkel',
       'Kinderporno', 'Kinderprostitutie'], dtype=object)

In [21]:
## Drop all rows were crime_cat is NaN after mapping
crime_df.dropna(subset=['crime_cat'], inplace=True)

In [22]:
crime_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1,crime_cat,gemeente,year
40248,590742,1.4.1,GM1680,2012JJ00,13.0,Zedendelicten,Aa en Hunze,2012
40249,590755,1.4.1,GM1680,2013JJ00,9.0,Zedendelicten,Aa en Hunze,2013
40250,590768,1.4.1,GM1680,2014JJ00,10.0,Zedendelicten,Aa en Hunze,2014
40251,590781,1.4.1,GM1680,2015JJ00,13.0,Zedendelicten,Aa en Hunze,2015
40252,590794,1.4.1,GM1680,2016JJ00,16.0,Zedendelicten,Aa en Hunze,2016
...,...,...,...,...,...,...,...,...
205707,2999528,3.2.2,GM0999,2020JJ00,NaN,Kinderprostitutie,Gemeenten; niet in te delen,2020
205708,2999541,3.2.2,GM0999,2021JJ00,NaN,Kinderprostitutie,Gemeenten; niet in te delen,2021
205709,2999554,3.2.2,GM0999,2022JJ00,NaN,Kinderprostitutie,Gemeenten; niet in te delen,2022
205710,2999567,3.2.2,GM0999,2023JJ00,NaN,Kinderprostitutie,Gemeenten; niet in te delen,2023


In [23]:
## Fill remaining NaN values in GeregistreerdeMisdrijven with 0
crime_df['GeregistreerdeMisdrijven_1'].fillna(value=0, inplace=True)

crime_df

C:\Users\chels\AppData\Local\Temp\ipykernel_23624\1183564811.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  crime_df['GeregistreerdeMisdrijven_1'].fillna(value=0, inplace=True)


,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1,crime_cat,gemeente,year
40248,590742,1.4.1,GM1680,2012JJ00,13.0,Zedendelicten,Aa en Hunze,2012
40249,590755,1.4.1,GM1680,2013JJ00,9.0,Zedendelicten,Aa en Hunze,2013
40250,590768,1.4.1,GM1680,2014JJ00,10.0,Zedendelicten,Aa en Hunze,2014
40251,590781,1.4.1,GM1680,2015JJ00,13.0,Zedendelicten,Aa en Hunze,2015
40252,590794,1.4.1,GM1680,2016JJ00,16.0,Zedendelicten,Aa en Hunze,2016
...,...,...,...,...,...,...,...,...
205707,2999528,3.2.2,GM0999,2020JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2020
205708,2999541,3.2.2,GM0999,2021JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2021
205709,2999554,3.2.2,GM0999,2022JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2022
205710,2999567,3.2.2,GM0999,2023JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2023


In [24]:
crime_df.isna().sum()

ID                            0
SoortMisdrijf                 0
RegioS                        0
Perioden                      0
GeregistreerdeMisdrijven_1    0
crime_cat                     0
gemeente                      0
year                          0
dtype: int64

## Import & extract population data

In [26]:
## Extracted from: https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=37230ned&_theme=270
## https://opendata.cbs.nl/statline/#/CBS/nl/dataset/37230ned/table?ts=1752318493150
## On: 12-07-2025

popdata = 'Bevolking_Gemeenten.csv'

popdata_df = pd.read_csv(popdata, sep=';')

popdata_df.head()

,ID,RegioS,Perioden,BevolkingAanHetBeginVanDePeriode_1,BevolkingAanHetEindeVanDePeriode_15
0,17340,GM1680,2002JJ00,25552.0,25305.0
1,17353,GM1680,2003JJ00,25305.0,25218.0
2,17366,GM1680,2004JJ00,25218.0,25329.0
3,17379,GM1680,2005JJ00,25329.0,25507.0
4,17392,GM1680,2006JJ00,25507.0,25563.0


In [27]:
## Keep only 'bevolking aan het einde van de periode', RegioS and Perioden
popdata_df.drop(labels=['ID','BevolkingAanHetBeginVanDePeriode_1'], axis=1, inplace=True)

In [28]:
## Rename bevolking column to PopYearEnd
popdata_df.rename(mapper={'BevolkingAanHetEindeVanDePeriode_15':'popyearend'}, axis=1, inplace=True)

In [29]:
popdata_df

,RegioS,Perioden,popyearend
0,GM1680,2002JJ00,25305.0
1,GM1680,2003JJ00,25218.0
2,GM1680,2004JJ00,25329.0
3,GM1680,2005JJ00,25507.0
4,GM1680,2006JJ00,25563.0
...,...,...,...
12783,GM0193,2020JJ00,129840.0
12784,GM0193,2021JJ00,130668.0
12785,GM0193,2022JJ00,132411.0
12786,GM0193,2023JJ00,133141.0


In [30]:
## Merge popdata_df to crime_df based on common RegioS and Perioden keys
crimepc_df = pd.merge(crime_df, popdata_df, on=['RegioS', 'Perioden'], how='left')

In [31]:
crimepc_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1,crime_cat,gemeente,year,popyearend
0,590742,1.4.1,GM1680,2012JJ00,13.0,Zedendelicten,Aa en Hunze,2012,25541.0
1,590755,1.4.1,GM1680,2013JJ00,9.0,Zedendelicten,Aa en Hunze,2013,25357.0
2,590768,1.4.1,GM1680,2014JJ00,10.0,Zedendelicten,Aa en Hunze,2014,25203.0
3,590781,1.4.1,GM1680,2015JJ00,13.0,Zedendelicten,Aa en Hunze,2015,25243.0
4,590794,1.4.1,GM1680,2016JJ00,16.0,Zedendelicten,Aa en Hunze,2016,25286.0
...,...,...,...,...,...,...,...,...,...
49187,2999528,3.2.2,GM0999,2020JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2020,NaN
49188,2999541,3.2.2,GM0999,2021JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2021,NaN
49189,2999554,3.2.2,GM0999,2022JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2022,NaN
49190,2999567,3.2.2,GM0999,2023JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2023,NaN


## Calculate per capita crime in each category & year, per gemeente

In [33]:
## Calculate new column crime_per_cap by dividing GeregistreerdeMisdrijven in each gemeente/category/year by per thousand of population per year/gemeente
crimepc_df['crime_per_thousand'] = crimepc_df['GeregistreerdeMisdrijven_1']/(crimepc_df['popyearend']/1000)

In [34]:
crimepc_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1,crime_cat,gemeente,year,popyearend,crime_per_thousand
0,590742,1.4.1,GM1680,2012JJ00,13.0,Zedendelicten,Aa en Hunze,2012,25541.0,0.508986
1,590755,1.4.1,GM1680,2013JJ00,9.0,Zedendelicten,Aa en Hunze,2013,25357.0,0.354932
2,590768,1.4.1,GM1680,2014JJ00,10.0,Zedendelicten,Aa en Hunze,2014,25203.0,0.396778
3,590781,1.4.1,GM1680,2015JJ00,13.0,Zedendelicten,Aa en Hunze,2015,25243.0,0.514994
4,590794,1.4.1,GM1680,2016JJ00,16.0,Zedendelicten,Aa en Hunze,2016,25286.0,0.632761
...,...,...,...,...,...,...,...,...,...,...
49187,2999528,3.2.2,GM0999,2020JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2020,NaN,NaN
49188,2999541,3.2.2,GM0999,2021JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2021,NaN,NaN
49189,2999554,3.2.2,GM0999,2022JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2022,NaN,NaN
49190,2999567,3.2.2,GM0999,2023JJ00,0.0,Kinderprostitutie,Gemeenten; niet in te delen,2023,NaN,NaN


In [36]:
crimepc_sub = crimepc_df[['crime_cat','gemeente','popyearend','crime_per_thousand']]
crimepc_sub

,crime_cat,gemeente,popyearend,crime_per_thousand
0,Zedendelicten,Aa en Hunze,25541.0,0.508986
1,Zedendelicten,Aa en Hunze,25357.0,0.354932
2,Zedendelicten,Aa en Hunze,25203.0,0.396778
3,Zedendelicten,Aa en Hunze,25243.0,0.514994
4,Zedendelicten,Aa en Hunze,25286.0,0.632761
...,...,...,...,...
49187,Kinderprostitutie,Gemeenten; niet in te delen,NaN,NaN
49188,Kinderprostitutie,Gemeenten; niet in te delen,NaN,NaN
49189,Kinderprostitutie,Gemeenten; niet in te delen,NaN,NaN
49190,Kinderprostitutie,Gemeenten; niet in te delen,NaN,NaN


In [42]:
## Calculate total violent crime per thousand, per gemeente

In [44]:
## Drop all rows where gemeente value is Buitenland or Gemeenten: niet in te delen

# Time series selected gemeenten of violent crime per capita

## Gemeentes with sharpest change in violent crime per capita (2012 -2024)

# Top gemeentes by violent crime per capita (2024)

## Homicides (moord, doodslag)

## Sex crime (zedendelicten, kinderporno, kinderprostitutie)